In [1]:
import tensorflow as tf
import numpy as np

2022-01-20 16:26:21.256090: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_19_11:
2022-01-20 16:26:21.256119: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class Segmenter():
    def __init__(self, model_path="../data/models/GloVe_w_LSTM", max_tokens=200):
        self.model = tf.keras.models.load_model(model_path)
        self.max_tokens = max_tokens
        

#     def predict(self, X):
#         preds = []
        
#         X_splitted = X.split()
#         n_tokens = len(X_splitted)
#         if n_tokens < self.max_tokens:
#             print("came here")
#             x_to_pred = np.array([' '.join(X_splitted)])
            
#             preds = self.model.predict(X).numpy().reshape(1,-1)
#             preds = (preds > 0.5).astype(int)
#         else:
#             i = 0
#             while i < n_tokens:
#                 print("no. It actually came here")
#                 print(i, n_tokens)
#                 x_sliced = X_splitted[i:i + self.max_tokens]
#                 x_to_pred = np.array([' '.join(x_sliced)])
                
#                 partial_preds = self.model.predict(x_to_pred).numpy().reshape(1,-1)
#                 partial_preds = (partial_preds > 0.5).astype(int)

#                 punctuations = np.argwhere(partial_preds == 1)
#                 # Shift all index by 1
#                 punctuations += 1
#                 if punctuations.shape[0] == 0:
#                     preds += [n_tokens]
#                     break
#                 else:
#                     punctuations = punctuations[:, 1].reshape(-1)
#                     last_punctuation = punctuations[-1]

#                 # Adds only the text until the last punctuation, because
#                 # otherwise the next phrase will lack the entire context
#                 punctuations += i
#                 preds += punctuations.tolist()

#                 i += last_punctuation
        
#         return preds

    def predict(self, X):
            preds = [0]

            X_splitted = X.split()
            n_tokens = len(X_splitted)
            i = 0
            while i < n_tokens:
                x_sliced = X_splitted[i:i + self.max_tokens]
                x_to_pred = np.array([' '.join(x_sliced)])

                partial_preds = self.model.predict(x_to_pred).numpy().reshape(1,-1)
                partial_preds = (partial_preds > 0.5).astype(int)

                punctuations = np.argwhere(partial_preds == 1)
                # Shift all index by 1
                punctuations += 1
                if punctuations.shape[0] == 0:
                    preds += [n_tokens]
                    break
                else:
                    punctuations = punctuations[:, 1].reshape(-1)
                    last_punctuation = punctuations[-1]

                # Adds only the text until the last punctuation, because
                # otherwise the next phrase will lack the entire context
                punctuations += i
                preds += punctuations.tolist()

                i += last_punctuation

            return preds
    
    def segment(self, X, punctuations):
        sentences = []
        
        X_splitted = X.split()
        for i in range(1, len(punctuations)):
            left = punctuations[i-1]
            right = punctuations[i]
            
            sentences.append(' '.join(X_splitted[left:right]))
        
        return sentences

In [3]:
segmenter = Segmenter()

2022-01-20 16:26:24.042306: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-20 16:26:24.042324: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kunumi): /proc/driver/nvidia/version does not exist
2022-01-20 16:26:24.042490: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
f = open("../data/01_preprocessed/without_inputation/-66g38EP0pI")

In [5]:
X = f.read()

In [6]:
segmenter.segment(X, segmenter.predict(X))

["hey I'm an angel and today we're gonna talk about Dunkirk which is director Christopher Nolan's 2017 war drama on MG and the echo Oh Bela and I had reacted to the trailer when it came out and I was surprised to see Christopher Nolan attacking war on attacking producing a war movie it didn't really seem to be a theme consistent with what he's been producing so far",
 "and I guess maybe I was expecting something like hacksaw Ridge but that's not at all we get",
 "so let's dive in this movie is dark sea",
 "I had recently watched the movie it and that didn't give me nightmares surprisingly Dunkirk gave me nightmares",
 'I had night terrors waking up with these war memories',
 'and I think I had a very tiny glimpse of what it must be like to be a veteran and suffer from PTSD',
 'and it is terrifying',
 'my heart was racing',
 'I felt like I was being bombed',
 'anyways this movie gets into your head if you let it',
 'and how can you not as a person who personally is terrified of the noti